In [1]:
#Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import time
import json
from pprint import pprint
from scipy.stats import ttest_ind
from datetime import datetime
from config import yelp_key

In [2]:
county_datafile = "data/LOS_ANGELES_COUNTY_RESTAURANT_AND_MARKET_INSPECTIONS.csv"

county_data = pd.read_csv(county_datafile)


In [3]:
columns = ['ACTIVITY DATE','FACILITY ID','FACILITY NAME','PROGRAM STATUS','PE DESCRIPTION','FACILITY ADDRESS','FACILITY CITY','FACILITY ZIP','SCORE','GRADE']
cleaned_county_data = county_data[columns]
cleaned_county_data['ACTIVITY DATE'] = pd.to_datetime(cleaned_county_data['ACTIVITY DATE'])
cleaned_county_data.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,ACTIVITY DATE,FACILITY ID,FACILITY NAME,PROGRAM STATUS,PE DESCRIPTION,FACILITY ADDRESS,FACILITY CITY,FACILITY ZIP,SCORE,GRADE
0,2016-06-22,FA0179753,TEA BAR,ACTIVE,RESTAURANT (31-60) SEATS HIGH RISK,954 E ALOSTA AVE,AZUSA,91702,93,A
1,2016-09-07,FA0239319,SHEFA,ACTIVE,RESTAURANT (0-30) SEATS HIGH RISK,7275 MELROSE AVE,LOS ANGELES,90046,90,A
2,2016-09-09,FA0071154,SUBWAY,ACTIVE,RESTAURANT (0-30) SEATS MODERATE RISK,28788 THE OLD RD,VALENCIA,91355,98,A
3,2016-04-01,FA0144494,AFC SUSHI @ VONS #3138,ACTIVE,RESTAURANT (0-30) SEATS HIGH RISK,16550 SOLEDAD CANYON RD,CANYON COUNTRY,91387-3215,99,A
4,2016-04-01,FA0005693,ALBERTS MEXICAN FOOD,INACTIVE,RESTAURANT (0-30) SEATS HIGH RISK,2355 PECK RD,SOUTH EL MONTE,91733,92,A


In [4]:
cleaned_county_data.sort_values(by = ["ACTIVITY DATE","FACILITY ID","FACILITY ADDRESS"],inplace = True,ascending=True)
cleaned_county_data.count()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


ACTIVITY DATE       199950
FACILITY ID         199950
FACILITY NAME       199950
PROGRAM STATUS      199950
PE DESCRIPTION      199950
FACILITY ADDRESS    199950
FACILITY CITY       199950
FACILITY ZIP        199950
SCORE               199950
GRADE               199895
dtype: int64

In [5]:
unique_county_data = cleaned_county_data.drop_duplicates(subset = ['FACILITY ID'], keep = 'last')
#print(unique_county_data.count())
unique_county_data = unique_county_data.loc[(unique_county_data['PROGRAM STATUS'] != "INACTIVE"),:]
#print(unique_county_data.count())
unique_county_data = unique_county_data[unique_county_data['PE DESCRIPTION'].str.contains("RESTAURANT")]
#print(unique_county_data.count())
unique_county_data = unique_county_data[~unique_county_data['FACILITY NAME'].str.contains("#")]
#print(unique_county_data.count())
unique_county_data = unique_county_data.dropna(how='any')
print(unique_county_data.count())

ACTIVITY DATE       22512
FACILITY ID         22512
FACILITY NAME       22512
PROGRAM STATUS      22512
PE DESCRIPTION      22512
FACILITY ADDRESS    22512
FACILITY CITY       22512
FACILITY ZIP        22512
SCORE               22512
GRADE               22512
dtype: int64


In [6]:
sample_data = unique_county_data.sample(n=100)
sample_data

,ACTIVITY DATE,FACILITY ID,FACILITY NAME,PROGRAM STATUS,PE DESCRIPTION,FACILITY ADDRESS,FACILITY CITY,FACILITY ZIP,SCORE,GRADE
172581,2018-06-07,FA0175096,PAPA JOHN'S PIZZA,ACTIVE,RESTAURANT (0-30) SEATS MODERATE RISK,2018 W K AVENUE,LANCASTER,93536,95,A
196884,2018-09-19,FA0244639,GONG CHA,ACTIVE,RESTAURANT (0-30) SEATS MODERATE RISK,3785 WILSHIRE BLVD # 107E,LOS ANGELES,90010,98,A
190959,2018-08-24,FA0262225,SQUARE ONE TAQUERIA,ACTIVE,RESTAURANT (0-30) SEATS HIGH RISK,4850 FOUNTAIN AVE,LOS ANGELES,90029,95,A
199841,2018-09-28,FA0243730,KREATIONS WEST HOLLYWOOD,ACTIVE,RESTAURANT (0-30) SEATS MODERATE RISK,8910 SANTA MONICA BLVD,WEST HOLLYWOOD,90069,95,A
183048,2018-07-24,FA0137949,MOMENT HOTEL,ACTIVE,RESTAURANT (0-30) SEATS HIGH RISK,7370 SUNSET BLVD,LOS ANGELES,90046,95,A
164531,2018-05-04,FA0256860,PIENCONE,ACTIVE,RESTAURANT (61-150) SEATS MODERATE RISK,1958 COLORADO BLVD,LOS ANGELES,90041,96,A
160722,2018-04-20,FA0252498,BUBBLE CRUSH,ACTIVE,RESTAURANT (0-30) SEATS MODERATE RISK,848 E GARVEY AVE UNIT D,MONTEREY PARK,91755,90,A
155859,2018-04-02,FA0241405,LA BONA ENERGIA,ACTIVE,RESTAURANT (0-30) SEATS MODERATE RISK,6515 VAN NUYS BLVD # D,VAN NUYS,91401,92,A
198920,2018-09-26,FA0011405,DOMINO'S PIZZA,ACTIVE,RESTAURANT (0-30) SEATS MODERATE RISK,15245 ROSCOE BLVD,PANORAMA CITY,91402,81,B
187477,2018-08-09,FA0025965,MYRA'S SALVADOREAN CUISINE,ACTIVE,RESTAURANT (31-60) SEATS HIGH RISK,7324 FLORENCE AVE,DOWNEY,90240,94,A


In [ ]:
#headers = {"Authorization": "Bearer "+yelp_key}
#params = {"term": unique_county_data['FACILITY NAME'].iloc[0],"location": unique_county_data['FACILITY ZIP'].iloc[0]}
#target_url = "https://api.yelp.com/v3/businesses/search"

#yelp_reviews = requests.get(target_url, params, headers=headers).json()
#yelp_reviews

In [7]:
headers = {"Authorization": "Bearer "+ yelp_key}
target_url = "https://api.yelp.com/v3/businesses/search"

final_df = pd.DataFrame()
name = []
rating = []
review_count = []
price = []

#for index, row in sample_data.iloc[:10,:].iterrows():
for index, row in sample_data.iterrows():
    restr_name = row['FACILITY NAME']
    restr_zip = row['FACILITY ZIP']
    restr_addr = row['FACILITY ADDRESS']
    restr_city = row['FACILITY CITY']
    params = {"term": restr_name,"location": str(restr_addr + ", " + restr_city + ", "+restr_zip)}
    
    yelp_response = requests.get(target_url, params, headers=headers)
    
    if yelp_response.status_code != requests.codes.ok:
        print(yelp_response.text)
    else:
        yelp_data = yelp_response.json()
        try:
            name.append(yelp_data['businesses'][0]['name'])
            #print(yelp_data['businesses'][0]['name'])
        except (KeyError, IndexError):
            name.append("NA")
            #print("Missing Name...")
        try:
            rating.append(yelp_data['businesses'][0]['rating'])
            #print(yelp_data['businesses'][0]['rating'])
        except (KeyError, IndexError):
            rating.append("NA")
            #print("Missing Rating...")
        try:
            review_count.append(yelp_data['businesses'][0]['review_count'])
            #print(yelp_data['businesses'][0]['review_count'])
        except (KeyError, IndexError):
            review_count.append("NA")
            #print("Missing Reviews...")    
        try:    
            price.append(yelp_data['businesses'][0]['price'])
            #print(yelp_data['businesses'][0]['price'])
        except (KeyError, IndexError):
            price.append("NA")
            #print("Missing Price...")


In [ ]:
final_df['Name'] = name
final_df['Inspection_Score'] = sample_data['SCORE'].tolist()
final_df['Inspection_Grade'] = sample_data['GRADE'].tolist()
final_df['Rating'] = rating
final_df['Reviews'] = review_count
final_df['Price'] = price

final_df.head()

In [ ]:
final_df['Reviews'] = pd.to_numeric(final_df['Reviews'],errors = 'coerce').fillna(0).astype(np.int64)
q1 = np.percentile(final_df['Reviews'],25)
q1

In [ ]:
percentiled_df = final_df.loc[final_df['Reviews'] > q1, :]
percentiled_df

In [ ]:
# Save Data to csv
percentiled_df.to_csv("final_output_data.csv")